In [1]:
import torch
import numpy as np

### RNN basics

In [ ]:
input_size = 4
hidden_size = 2

# One-Hot Encoding
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]
input_data_np = np.array([[h, e, l, l, o], 
						  [e, o, l, l, l], 
						  [l, l, e, e, l]], dtype=np.float32)
input_data = torch.Tensor(input_data_np)

rnn = torch.nn.RNN(input_size, hidden_size)
outputs, _status = rnn(input_data)

### RNN hihello 

In [7]:
### hihello
char_set = ["h", "i", "e", "l", "o"]

# hyper params
input_size = len(char_set)
hidden_size = len(char_set)
learning_rate = 0.1

# data setting
x_data = [[0, 1, 0, 2, 3, 3]]
x_one_hot = [[[1, 0, 0, 0, 0], 
              [0, 1, 0, 0, 0], 
              [1, 0, 0, 0, 0], 
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0], 
              [0, 0, 0, 1, 0]]]
y_data = [[1, 0, 2, 3, 3, 4]]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

### RNN charseq

In [24]:
### charseq
sample = " if you want you"

# make dictionary
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}

# hyper params
dic_size = len(char_dic)
input_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1


# data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)


# declare RNN
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # batch_first guarantees the order of output = (B, S, F)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), learning_rate)

# start training
for i in range(100):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    result = outputs.data.numpy().argmax(axis=2)
    result_str = "".join([char_set[c] for c in np.squeeze(result)])
    print(f"Training {i}") 
    print("     loss:           ", loss.item())
    print("     prediction:     ", result) 
    print("     true Y:         ", y_data)
    print("     prediction str: ", result_str)

Training 0
     loss:            2.4721407890319824
     prediction:      [[2 2 3 2 3 3 2 3 3 3 2 2 3 3 3]]
     true Y:          [[4, 0, 1, 8, 5, 2, 1, 6, 7, 9, 3, 1, 8, 5, 2]]
     prediction str:  uututtutttuuttt
Training 1
     loss:            2.1711106300354004
     prediction:      [[4 5 2 4 5 2 8 4 4 8 8 9 4 5 2]]
     true Y:          [[4, 0, 1, 8, 5, 2, 1, 6, 7, 9, 3, 1, 8, 5, 2]]
     prediction str:  iouiouyiiyyniou
Training 2
     loss:            1.9528337717056274
     prediction:      [[8 5 1 1 5 2 1 1 1 1 1 1 1 5 2]]
     true Y:          [[4, 0, 1, 8, 5, 2, 1, 6, 7, 9, 3, 1, 8, 5, 2]]
     prediction str:  yo  ou       ou
Training 3
     loss:            1.8191578388214111
     prediction:      [[1 5 1 1 5 2 1 5 7 1 1 1 1 5 2]]
     true Y:          [[4, 0, 1, 8, 5, 2, 1, 6, 7, 9, 3, 1, 8, 5, 2]]
     prediction str:   o  ou oa    ou
Training 4
     loss:            1.6708742380142212
     prediction:      [[8 5 1 1 5 2 1 1 1 1 1 1 1 5 2]]
     true Y:          [[4, 0

### RNN longseq

In [34]:
sentence = ("if you want to bulid a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.")

# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

# hyper params
dic_size = len(char_dic)
input_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10 # window
learning_rate = 0.1

# data setting
x_data = []
y_data = []

for i in range(0, len(sentence)-sequence_length):
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1 : i+sequence_length+1]
    
    # print(i, x_str, "->", y_str)
    
    x_data.append([char_dic[c] for c in x_str]) # x_str to index
    y_data.append([char_dic[c] for c in y_str]) # y_str to index
    
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data) 

# declare RNN + FC 
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(
            input_dim, 
            hidden_dim, 
            num_layers=layers, 
            batch_first=True
            )
        self.fc = torch.nn.Linear(
            hidden_dim, 
            hidden_dim, 
            bias=True
            )
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

net = Net(dic_size, hidden_size, 2)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), learning_rate)

# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        print(i, j, "".join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += "".join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]
            
    # print(f"Training {i}") 
    # print("     loss:           ", loss.item())
    # print("     prediction:     ", result) 
    # print("     true Y:         ", y_data)
    # print("     prediction str: ", result_str)

0 0 niinnniinn 3.2409441471099854
0 1 iinnniinni 3.2409441471099854
0 2 innniinnii 3.2409441471099854
0 3 nnniinniig 3.2409441471099854
0 4 nniinniign 3.2409441471099854
0 5 niiniiigni 3.2409441471099854
0 6 iiniiignin 3.2409441471099854
0 7 inniigninc 3.2409441471099854
0 8 nniigninci 3.2409441471099854
0 9 iiignincin 3.2409441471099854
0 10 iignincinn 3.2409441471099854
0 11 innigcinnn 3.2409441471099854
0 12 inigcinnnn 3.2409441471099854
0 13 nincinnnnn 3.2409441471099854
0 14 inninnnnni 3.2409441471099854
0 15 nnininnnin 3.2409441471099854
0 16 nininnninn 3.2409441471099854
0 17 iniinninni 3.2409441471099854
0 18 nnnniinnii 3.2409441471099854
0 19 niniinniii 3.2409441471099854
0 20 inninniiig 3.2409441471099854
0 21 ninfciiign 3.2409441471099854
0 22 innciiignc 3.2409441471099854
0 23 innigigncn 3.2409441471099854
0 24 inigigncnc 3.2409441471099854
0 25 ninigncici 3.2409441471099854
0 26 niigncicig 3.2409441471099854
0 27 iinncicign 3.2409441471099854
0 28 inncicignn 3.240944147109